In [1]:
import theano
import theano.tensor as tt
import numpy as np
import pandas as pd
import pickle as pkl
import datetime
from collections import OrderedDict
import cProfile
import pstats
%run ../src/sampling_utils.py
%run ../src/shared_utils.py

The always stuff

In [2]:
with open('../data/counties/counties.pkl', "rb") as f:
    counties = pkl.load(f)
    
disease = "covid19"
prediction_region = "germany"

indata = load_daily_data(disease, prediction_region, counties)
data = indata.iloc[:,0:31]
#data = indata.iloc[:,0:31]
rnd_tsel = np.random.Generator(np.random.PCG64(12345))
times_by_day = uniform_times_by_day(data.index, rnd_tsel)

rnd_csel = np.random.Generator(np.random.PCG64(12345))
locations_by_county=uniform_locations_by_county(counties, rnd_csel)

The old stuff

In [3]:
def old_sample_time_and_space(data, times_by_day, locations_by_county, rnd_t, rnd_l):
    n_total = data.sum().sum()
    t_all = np.empty((n_total,), dtype=object)
    x_all = np.empty((n_total, 2))
    
    i=0
    for (county_id, series) in data.iteritems():
        for (day, n) in series.iteritems():
            #if n==0: continue
            #print(i,"\n   day =",day,"\n   no. samples to draw = ",n)
        
            # draw n random times
            times = times_by_day[day]
            #idx = rnd_time.choice(len(times), n)
            idx = np.floor( (n*[len(times)]) * rnd_t.random((n,)) ).astype("int32") # replace 'rnd_time.choice' to enable compare with new optimized solution
            #print("   random sample ids   = ",idx)
            t_all[i:i + n] = times[idx]

            # draw n random locations
            locs = locations_by_county[county_id]
            #idx = rnd_loc.choice(locs.shape[0], n)
            idx = np.floor( (n*[locs.shape[0]]) * rnd_l.random((n,)) ).astype("int32") # replace 'rnd_time.choice' to enable compare with new optimized solution
            x_all[i:i + n, :] = locs[idx, :]
        
            i += n          

    return t_all, x_all


ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs, profile=False)

def sample_ia_effects_old():
    rnd_time = np.random.Generator(np.random.PCG64(12345))
    rnd_loc  = np.random.Generator(np.random.PCG64(12345))
    rnd_time_pred = np.random.Generator(np.random.PCG64(12345))
    rnd_loc_pred  = np.random.Generator(np.random.PCG64(12345))

    t_data_0 = []
    x_data_0 = []
    t_pred_0 = []
    x_pred_0 = []

    num_tps=5
    d_offs=0 # just to limit the time of test
    c_offs=0 # just to limit the time of test
    days = data.index[d_offs:d_offs+50]
    counties = data.columns[c_offs:c_offs+50]

    _to_timestamp = np.frompyfunc(datetime.datetime.timestamp, 1, 1)
    num_features = len(temporal_bfs(tt.fmatrix("tmp"))) * len(spatial_bfs(tt.fmatrix("tmp")))
    res_0 = np.zeros((len(days), len(counties), num_features), dtype=np.float32)

    for i, day in enumerate(days):
        for j, county in enumerate(counties):
            idx = ((day - pd.Timedelta(days=5)) <= data.index) * (data.index < day)

            t_data, x_data = old_sample_time_and_space(data.iloc[idx], times_by_day, locations_by_county, rnd_time, rnd_loc)
            t_pred, x_pred = old_sample_time_and_space(pd.DataFrame(num_tps, index=[day], columns=[county]), times_by_day, locations_by_county, rnd_time_pred, rnd_loc_pred)

            #print("_to_timestamp(t_pred) (types, type1, size, value): ", type(_to_timestamp(t_pred)), type(_to_timestamp(t_pred)[0]), np.shape(_to_timestamp(t_pred)), _to_timestamp(t_pred)[0])
            # => _to_timestamp(t_pred) (types, type1, size, value):  <class 'numpy.ndarray'> <class 'float'> (5,) 1580217693.876309
            #print("x_pred (types, size, value)       : ", type(x_pred), type(x_pred[0]), type(x_pred[0][0]), np.shape(x_pred), x_pred[0][0])        
            # => x_pred (types, size, value)       :  <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.float64'> (5, 2) 10.134378974970323

            res_0[i, j, :] = ia_bfs(_to_timestamp(t_pred), x_pred, _to_timestamp(t_data), x_data)        

            # store all to compare with old algo
            t_data_0 = t_data_0 + t_data.tolist()
            x_data_0 = x_data_0 + x_data.tolist()
            t_pred_0 = t_pred_0 + t_pred.tolist()
            x_pred_0 = x_pred_0 + x_pred.tolist()
            
    return t_data_0, x_data_0, t_pred_0, x_pred_0

C:\Users\Jomann\anaconda3\lib\site-packages\theano\tensor\subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
C:\Users\Jomann\anaconda3\lib\site-packages\theano\tensor\subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


In [4]:
%%timeit
t_data_0, x_data_0, t_pred_0, x_pred_0 = sample_ia_effects_old()


15 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The new stuff

In [ ]:

#ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs, profile=False)
#%%timeit
#cp = cProfile.Profile()
#cp.enable()
samp = iaeffect_sampler(data, times_by_day, locations_by_county, temporal_bfs, spatial_bfs)
#cp.disable()
#cp.create_stats()
#p = pstats.Stats()
#p.strip_dirs()
#cp.dump_stats('profilenumbasingle.dmp')

In [ ]:
#print(np.shape(data))
#print(np.shape(data.index))
#print(np.shape(data.columns))
#print(type(samp), type(samp[0]), type(samp[0][0]), type(samp[0][0][0]))
#print(not 5)